# Feature Engineering - Pipeline MLOps

Este notebook implementa la **Ingeniería de Características** para el proyecto de predicción de Alzheimer.

**🔍 Propósito de este Notebook:**
- Este notebook es **AUTOCONTENIDO** y puede ejecutarse de forma independiente
- Muestra de forma manual y gráfica el proceso de Feature Engineering
- No depende de scripts externos ni de pasos anteriores
- Ideal para revisión, comprensión y experimentación

**Funcionalidades:**
- Carga de datos directamente desde el CSV original
- Limpieza básica de datos (eliminar IDs, duplicados)
- Creación de features derivados basados en el análisis EDA
- Clasificación automática de tipos de variables
- Construcción de pipelines de preprocesamiento (sklearn)
- Transformaciones: imputación, escalado, codificación
- Separación train-test estratificada
- Visualizaciones del proceso de transformación
- Guardado de artefactos (preprocessor, datasets transformados)

**📋 Diferencia con los Scripts:**
- **Notebooks (`/notebooks/`)**: Revisión manual paso a paso, con visualizaciones y explicaciones
- **Scripts (`/scripts/`)**: Automatización para ejecución completa del pipeline y Docker

**Basado en hallazgos del EDA:** `comprension_eda.ipynb` (Sección 8.7)

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import json
import os
import joblib
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Sklearn para feature engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Configuración visual
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Librerías importadas correctamente")
print(f"✓ Pandas versión: {pd.__version__}")
print(f"✓ Numpy versión: {np.__version__}")

✓ Librerías importadas correctamente
✓ Pandas versión: 1.5.3
✓ Numpy versión: 1.24.3


## 1. Cargar Datos Originales y Configuración

Cargamos el dataset original directamente desde el CSV y la configuración del proyecto.

**Nota:** Este notebook NO depende de pasos anteriores. Carga los datos originales y realiza toda la limpieza necesaria.

In [2]:
# Cargar configuración
config_path = "../../../config.json"

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("✓ Configuración cargada desde config.json")
else:
    config = {"training": {"test_size": 0.2, "random_state": 42}}
    print("⚠️ config.json no encontrado. Usando configuración por defecto")

# Extraer parámetros de entrenamiento
test_size = config.get('training', {}).get('test_size', 0.2)
random_state = config.get('training', {}).get('random_state', 42)

print(f"\n📊 Parámetros de configuración:")
print(f"   • Test size: {test_size}")
print(f"   • Random state: {random_state}")

✓ Configuración cargada desde config.json

📊 Parámetros de configuración:
   • Test size: 0.2
   • Random state: 42


In [3]:
# Cargar dataset ORIGINAL (no procesado)
# Buscar primero desde config.json, sino usar el path por defecto

data_path = None

# Intentar cargar desde config
if os.path.exists(config_path):
    data_path = config.get('data_path_notebooks', "../../../alzheimers_disease_data.csv")
else:
    data_path = "../../../alzheimers_disease_data.csv"

# Cargar dataset original
if not os.path.exists(data_path):
    print("❌ ERROR: No se encontró el archivo de datos")
    print(f"   Buscado en: {data_path}")
    print("   Por favor, verifica que el archivo CSV existe en la ruta correcta")
else:
    df_raw = pd.read_csv(data_path)
    print(f"✓ Dataset ORIGINAL cargado desde: {data_path}")
    print(f"  Dimensiones: {df_raw.shape[0]} filas × {df_raw.shape[1]} columnas")
    print(f"\n📋 Primeras filas del dataset ORIGINAL:")
    display(df_raw.head())

✓ Dataset ORIGINAL cargado desde: ../../../alzheimers_disease_data.csv
  Dimensiones: 2149 filas × 35 columnas

📋 Primeras filas del dataset ORIGINAL:


,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid


## 1.5 Limpieza Básica de Datos

Realizamos limpieza básica antes del feature engineering:
- Eliminar columnas de identificación (PatientID, DoctorInCharge)
- Eliminar duplicados
- Verificar tipos de datos

In [15]:
print("="*80)
print("LIMPIEZA BÁSICA DE DATOS")
print("="*80 + "\n")

# Crear copia para no modificar el original
df = df_raw.copy()

print(f"📊 Dataset original: {df.shape[0]} filas × {df.shape[1]} columnas")

# 1. Eliminar duplicados
n_duplicates = df.duplicated().sum()
if n_duplicates > 0:
    df = df.drop_duplicates()
    print(f"✓ Eliminados {n_duplicates} registros duplicados")
else:
    print("✓ No se encontraron duplicados")

# 2. Eliminar columnas de identificación (no son features predictivos)
id_columns = ['PatientID', 'DoctorInCharge']
existing_id_cols = [col for col in id_columns if col in df.columns]

if existing_id_cols:
    df = df.drop(columns=existing_id_cols)
    print(f"✓ Eliminadas columnas de identificación: {existing_id_cols}")

# 3. Información sobre valores faltantes (se manejarán en el pipeline)
missing_info = df.isnull().sum()
if missing_info.sum() > 0:
    print(f"\n📊 Valores faltantes detectados (se imputarán en el pipeline):")
    for col, count in missing_info[missing_info > 0].items():
        pct = (count / len(df)) * 100
        print(f"   {col}: {count} ({pct:.2f}%)")
else:
    print("\n✓ No hay valores faltantes")

# 4. Verificar tipos de datos
print(f"\n📋 Tipos de datos:")
numeric_cols_check = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols_check = df.select_dtypes(include=['object']).columns.tolist()
print(f"   Numéricas: {len(numeric_cols_check)} columnas")
print(f"   Categóricas: {len(categorical_cols_check)} columnas")

print(f"\n✅ Limpieza completada")
print(f"   Dimensiones finales: {df.shape[0]} filas × {df.shape[1]} columnas")

print("\n" + "="*80)

LIMPIEZA BÁSICA DE DATOS

📊 Dataset original: 2149 filas × 35 columnas
✓ No se encontraron duplicados
✓ Eliminadas columnas de identificación: ['PatientID', 'DoctorInCharge']

✓ No hay valores faltantes

📋 Tipos de datos:
   Numéricas: 33 columnas
   Categóricas: 0 columnas

✅ Limpieza completada
   Dimensiones finales: 2149 filas × 33 columnas



## 2. Creación de Features Derivados

Basado en el análisis EDA (`comprension_eda.ipynb` Sección 8.7), implementamos los features derivados más relevantes.

## 2.1 Justificación Teórica de Features Derivados

Basado en literatura médica y hallazgos del EDA (`comprension_eda.ipynb`), implementamos los siguientes features con sus respectivas justificaciones:

### 🏥 Indicadores Cardiovasculares
**Ratios de Colesterol (LDL/HDL, Total/HDL)**
- **Justificación Clínica**: La relación entre colesterol LDL ("malo") y HDL ("bueno") es un indicador establecido de riesgo cardiovascular
- **Relevancia Alzheimer**: Estudios epidemiológicos demuestran asociación entre perfil lipídico y deterioro cognitivo
- **Ventaja sobre features brutos**: Captura relación no-lineal más relevante que valores absolutos
- **Referencia**: Framingham Heart Study, relación entre vascular risk y demencia

**Presión Arterial Media (MAP)**
- **Fórmula**: MAP = Diastolic + (Systolic - Diastolic) / 3
- **Justificación**: MAP es mejor indicador de perfusión cerebral que presión aislada
- **Relevancia**: Hipoperfusión cerebral vinculada a neurodegeneración
- **Ventaja**: Combina información de systolic y diastolic en una métrica fisiológicamente significativa

### 👶 Interacciones Edad
**Age_Squared (Edad²)**
- **Justificación**: La relación entre edad y Alzheimer no es lineal; riesgo aumenta exponencialmente
- **Captura**: Efectos no-lineales del envejecimiento en patología amiloide
- **Ventaja**: Permite al modelo aprender relaciones cuadráticas sin transformación explícita

**Age × Family History**
- **Justificación**: Interacción multiplicativa entre edad y predisposición genética
- **Relevancia**: El riesgo genético tiene mayor impacto a edades avanzadas
- **Ventaja**: Captura sinergia entre dos factores de riesgo

### 🔗 Score de Riesgo Cardiovascular
**CV_Risk_Score (Suma de condiciones CV)**
- **Justificación**: Índice agregado de comorbilidades cardiovasculares
- **Componentes**: CardiovascularDisease, Diabetes, Hypertension (factores de riesgo establecidos)
- **Ventaja**: Síntesis de múltiples condiciones en una métrica de riesgo
- **Clínico**: Refleja "carga de morbilidad cardiovascular"

### 📊 Basado en EDA
Todos estos features se derivaron del análisis en `comprension_eda.ipynb` Sección 8.7, donde se identificaron:
- Correlaciones significativas con diagnóstico
- Patrones no-lineales en la data
- Interacciones clínicamente relevantes
- Variables con mayor poder predictivo


In [5]:
def create_derived_features(df):
    """
    Crea features derivados basados en análisis EDA y literatura médica.
    
    ╔══════════════════════════════════════════════════════════════════════╗
    ║ FEATURES DERIVADOS Y SU JUSTIFICACIÓN MÉDICA/CLÍNICA               ║
    ╚══════════════════════════════════════════════════════════════════════╝
    
    1. Cholesterol_Ratio_LDL_HDL (LDL/HDL)
       • Justificación: Indicador establecido de riesgo cardiovascular
       • Relevancia AD: Lipid profile vinculado a deterioro cognitivo
       • Ventaja: Captura relación no-lineal vs valores absolutos
       • Ref: Framingham Heart Study
    
    2. Cholesterol_Total_HDL_Ratio (Total/HDL)
       • Justificación: Otro índice de riesgo cardiovascular
       • Clínico: Predictor independiente de enfermedad vascular
       • Relevancia: Vascular factors afectan patología amiloide
    
    3. Mean_Arterial_Pressure (MAP) = Diastolic + (Systolic-Diastolic)/3
       • Justificación: MAP mejor indicador de perfusión cerebral
       • Clínico: Hipoperfusión = neurodegeneración
       • Ventaja: Combina info systolic y diastolic en 1 métrica
    
    4. Age_Squared (Age²)
       • Justificación: Relación edad-Alzheimer NO es lineal
       • Captura: Riesgo aumenta exponencialmente con edad
       • ML: Permite modelo aprender relaciones cuadráticas
    
    5. Age_FH_Interaction (Age × FamilyHistoryAlzheimers)
       • Justificación: Interacción multiplicativa edad-genética
       • Clínico: Predisposición genética tiene mayor impacto en edades
       • Ventaja: Captura sinergia de dos factores de riesgo
    
    6. CV_Risk_Score (suma de condiciones cardiovasculares)
       • Justificación: Índice agregado de comorbilidades CV
       • Componentes: CardiovascularDisease, Diabetes, Hypertension
       • Clínico: Refleja "carga de morbilidad cardiovascular"
    
    Args:
        df: DataFrame original con variables clínicas
        
    Returns:
        DataFrame con 6 nuevos features derivados agregados
        
    Nota: Derivados de análisis EDA en comprension_eda.ipynb Sección 8.7
    """
    df_new = df.copy()
    features_created = []
    
    print("="*80)
    print("CREANDO FEATURES DERIVADOS")
    print("="*80 + "\n")
    
    # 1. Ratio de Colesterol LDL/HDL
    if 'CholesterolLDL' in df.columns and 'CholesterolHDL' in df.columns:
        df_new['Cholesterol_Ratio_LDL_HDL'] = df_new['CholesterolLDL'] / df_new['CholesterolHDL']
        features_created.append('Cholesterol_Ratio_LDL_HDL')
        print("✓ Creado: Cholesterol_Ratio_LDL_HDL (LDL/HDL)")
    
    # 2. Ratio de Colesterol Total/HDL
    if 'CholesterolTotal' in df.columns and 'CholesterolHDL' in df.columns:
        df_new['Cholesterol_Total_HDL_Ratio'] = df_new['CholesterolTotal'] / df_new['CholesterolHDL']
        features_created.append('Cholesterol_Total_HDL_Ratio')
        print("✓ Creado: Cholesterol_Total_HDL_Ratio (Total/HDL)")
    
    # 3. Presión Arterial Media (MAP)
    if 'SystolicBP' in df.columns and 'DiastolicBP' in df.columns:
        df_new['Mean_Arterial_Pressure'] = (
            df_new['DiastolicBP'] + (df_new['SystolicBP'] - df_new['DiastolicBP']) / 3
        )
        features_created.append('Mean_Arterial_Pressure')
        print("✓ Creado: Mean_Arterial_Pressure (MAP)")
    
    # 4. Edad al cuadrado
    if 'Age' in df.columns:
        df_new['Age_Squared'] = df_new['Age'] ** 2
        features_created.append('Age_Squared')
        print("✓ Creado: Age_Squared (Age²)")
    
    # 5. Interacción Edad x Historia Familiar
    if 'Age' in df.columns and 'FamilyHistoryAlzheimers' in df.columns:
        df_new['Age_FH_Interaction'] = df_new['Age'] * df_new['FamilyHistoryAlzheimers']
        features_created.append('Age_FH_Interaction')
        print("✓ Creado: Age_FH_Interaction (Age × FamilyHistory)")
    
    # 6. Score de riesgo cardiovascular (suma de condiciones)
    cv_conditions = ['CardiovascularDisease', 'Diabetes', 'Hypertension']
    if all(col in df.columns for col in cv_conditions):
        df_new['CV_Risk_Score'] = df_new[cv_conditions].sum(axis=1)
        features_created.append('CV_Risk_Score')
        print("✓ Creado: CV_Risk_Score (suma de condiciones cardiovasculares)")
    
    # Manejar valores infinitos o NaN resultantes
    for col in features_created:
        # Reemplazar infinitos con NaN
        df_new[col].replace([np.inf, -np.inf], np.nan, inplace=True)
        
        # Contar NaNs
        n_nan = df_new[col].isna().sum()
        if n_nan > 0:
            print(f"   ⚠️ {col}: {n_nan} valores NaN (serán imputados en el pipeline)")
    
    print(f"\n✅ Total de features derivados creados: {len(features_created)}")
    print(f"   Dimensiones nuevas: {df_new.shape[0]} filas × {df_new.shape[1]} columnas")
    print(f"   Features agregados: {len(features_created)}")
    
    return df_new

# Aplicar transformación
df_with_features = create_derived_features(df)

print("\n" + "="*80)

CREANDO FEATURES DERIVADOS

✓ Creado: Cholesterol_Ratio_LDL_HDL (LDL/HDL)
✓ Creado: Cholesterol_Total_HDL_Ratio (Total/HDL)
✓ Creado: Mean_Arterial_Pressure (MAP)
✓ Creado: Age_Squared (Age²)
✓ Creado: Age_FH_Interaction (Age × FamilyHistory)
✓ Creado: CV_Risk_Score (suma de condiciones cardiovasculares)

✅ Total de features derivados creados: 6
   Dimensiones nuevas: 2149 filas × 39 columnas
   Features agregados: 6



## 3. Clasificación de Tipos de Variables

Identificamos y clasificamos las variables en numéricas, categóricas nominales y categóricas ordinales.

In [16]:
# Definir variable objetivo
target_col = 'Diagnosis'

print("="*80)
print("CLASIFICACIÓN DE CARACTERÍSTICAS")
print("="*80 + "\n")

# Separar features del target
if target_col in df_with_features.columns:
    feature_cols = [col for col in df_with_features.columns if col != target_col]
    df_features = df_with_features[feature_cols]
    print(f"✓ Variable objetivo identificada: {target_col}")
    print(f"✓ Total de features: {len(feature_cols)}")
else:
    print(f"⚠️ Variable objetivo '{target_col}' no encontrada")
    df_features = df_with_features
    feature_cols = df_features.columns.tolist()

# Detectar tipos automáticamente
numeric_features = df_features.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df_features.select_dtypes(include=['object']).columns.tolist()

# Para este dataset, todas las categóricas son nominales
# Si hubiera ordinales específicas (ej: 'EducationLevel'), se definirían aquí
ordinal_features = []
nominal_features = categorical_features

print(f"\n📊 Variables Numéricas ({len(numeric_features)}):")
if numeric_features:
    for i, col in enumerate(numeric_features[:10], 1):
        print(f"   {i}. {col}")
    if len(numeric_features) > 10:
        print(f"   ... y {len(numeric_features) - 10} más")

print(f"\n📝 Variables Categóricas Nominales ({len(nominal_features)}):")
if nominal_features:
    for i, col in enumerate(nominal_features, 1):
        print(f"   {i}. {col}")
else:
    print("   Ninguna")

print(f"\n📈 Variables Categóricas Ordinales ({len(ordinal_features)}):")
if ordinal_features:
    for i, col in enumerate(ordinal_features, 1):
        print(f"   {i}. {col}")
else:
    print("   Ninguna")

print("\n" + "="*80)

CLASIFICACIÓN DE CARACTERÍSTICAS

✓ Variable objetivo identificada: Diagnosis
✓ Total de features: 38

📊 Variables Numéricas (38):
   1. Age
   2. Gender
   3. Ethnicity
   4. EducationLevel
   5. BMI
   6. Smoking
   7. AlcoholConsumption
   8. PhysicalActivity
   9. DietQuality
   10. SleepQuality
   ... y 28 más

📝 Variables Categóricas Nominales (0):
   Ninguna

📈 Variables Categóricas Ordinales (0):
   Ninguna



## 4. Construcción de Pipelines de Preprocesamiento

Creamos pipelines específicos para cada tipo de variable usando sklearn `Pipeline` y `ColumnTransformer`.

In [18]:
print("="*80)
print("CONSTRUCCIÓN DE PIPELINES DE PREPROCESAMIENTO")
print("="*80 + "\n")

transformers_list = []

# Pipeline para variables numéricas
if numeric_features:
    numeric_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    transformers_list.append(('numeric', numeric_pipeline, numeric_features))
    
    print(f"✓ Pipeline Numérico ({len(numeric_features)} features):")
    print(f"    1. SimpleImputer(strategy='median') - Imputa valores faltantes con la mediana")
    print(f"    2. StandardScaler() - Normaliza con media=0 y std=1")
    print()

# Pipeline para variables categóricas nominales
if nominal_features:
    nominal_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    transformers_list.append(('nominal', nominal_pipeline, nominal_features))
    
    print(f"✓ Pipeline Categórico Nominal ({len(nominal_features)} features):")
    print(f"    1. SimpleImputer(strategy='most_frequent') - Imputa con el valor más frecuente")
    print(f"    2. OneHotEncoder(handle_unknown='ignore') - Codificación one-hot")
    print()

# Pipeline para variables categóricas ordinales
if ordinal_features:
    ordinal_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ])
    transformers_list.append(('ordinal', ordinal_pipeline, ordinal_features))
    
    print(f"✓ Pipeline Categórico Ordinal ({len(ordinal_features)} features):")
    print(f"    1. SimpleImputer(strategy='most_frequent')")
    print(f"    2. OrdinalEncoder(handle_unknown='use_encoded_value')")
    print()

# Crear ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=transformers_list,
    remainder='drop'  # Eliminar columnas no especificadas
)

print(f"✅ ColumnTransformer creado con {len(transformers_list)} transformadores")
print(f"   • Transformadores: {[t[0] for t in transformers_list]}")
print(f"   • Remainder: 'drop' (columnas no especificadas serán eliminadas)")

print("\n" + "="*80)

CONSTRUCCIÓN DE PIPELINES DE PREPROCESAMIENTO

✓ Pipeline Numérico (38 features):
    1. SimpleImputer(strategy='median') - Imputa valores faltantes con la mediana
    2. StandardScaler() - Normaliza con media=0 y std=1

✅ ColumnTransformer creado con 1 transformadores
   • Transformadores: ['numeric']
   • Remainder: 'drop' (columnas no especificadas serán eliminadas)



## 5. Separación Train-Test

Separamos los datos en conjuntos de entrenamiento y evaluación con estratificación.

In [8]:
print("="*80)
print("SEPARACIÓN DE DATOS TRAIN-TEST")
print("="*80 + "\n")

# Separar features (X) y target (y)
X = df_with_features.drop(columns=[target_col])
y = df_with_features[target_col]

print(f"✓ Features (X): {X.shape}")
print(f"✓ Target (y): {y.shape}")

# Train-test split con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=random_state,
    stratify=y  # Mantener proporción de clases
)

print(f"\n📊 División train-test ({int((1-test_size)*100)}-{int(test_size*100)}):")
print(f"   Entrenamiento: {X_train.shape[0]:,} muestras ({(X_train.shape[0]/len(y)*100):.1f}%)")
print(f"   Evaluación:    {X_test.shape[0]:,} muestras ({(X_test.shape[0]/len(y)*100):.1f}%)")

# Verificar distribución de clases
print(f"\n📈 Distribución del target en ENTRENAMIENTO:")
train_dist = y_train.value_counts().sort_index()
for label, count in train_dist.items():
    print(f"   Clase {label}: {count:,} ({count/len(y_train)*100:.1f}%)")

print(f"\n📈 Distribución del target en EVALUACIÓN:")
test_dist = y_test.value_counts().sort_index()
for label, count in test_dist.items():
    print(f"   Clase {label}: {count:,} ({count/len(y_test)*100:.1f}%)")

# Verificar que la estratificación funcionó correctamente
print("\n✓ Estratificación exitosa: proporciones similares entre train y test")

print("\n" + "="*80)

SEPARACIÓN DE DATOS TRAIN-TEST

✓ Features (X): (2149, 38)
✓ Target (y): (2149,)

📊 División train-test (80-20):
   Entrenamiento: 1,719 muestras (80.0%)
   Evaluación:    430 muestras (20.0%)

📈 Distribución del target en ENTRENAMIENTO:
   Clase 0: 1,111 (64.6%)
   Clase 1: 608 (35.4%)

📈 Distribución del target en EVALUACIÓN:
   Clase 0: 278 (64.7%)
   Clase 1: 152 (35.3%)

✓ Estratificación exitosa: proporciones similares entre train y test



## 6. Ajuste y Transformación de Datos

Ajustamos el preprocessor SOLO con datos de entrenamiento (evitar data leakage) y transformamos ambos conjuntos.

In [9]:
print("="*80)
print("AJUSTE Y TRANSFORMACIÓN")
print("="*80 + "\n")

print("📊 Ajustando preprocessor con datos de entrenamiento...")
print("   (IMPORTANTE: Solo usar train para evitar data leakage)")
preprocessor.fit(X_train)
print("✓ Preprocessor ajustado exitosamente")

print("\n🔄 Transformando datos de entrenamiento...")
X_train_transformed = preprocessor.transform(X_train)
print(f"✓ X_train transformado: {X_train_transformed.shape}")

print("\n🔄 Transformando datos de evaluación...")
X_test_transformed = preprocessor.transform(X_test)
print(f"✓ X_test transformado: {X_test_transformed.shape}")

print(f"\n📈 Resumen de transformación:")
print(f"   Features originales:    {X_train.shape[1]}")
print(f"   Features transformados: {X_train_transformed.shape[1]}")
print(f"   Diferencia: {X_train_transformed.shape[1] - X_train.shape[1]:+d}")

# Explicar el aumento en features (si aplica)
if X_train_transformed.shape[1] > X_train.shape[1]:
    print(f"\n💡 Nota: El aumento de features se debe a:")
    print(f"   • OneHotEncoder crea una columna por cada categoría")
    print(f"   • Variables categóricas: {len(nominal_features)}")
    
print("\n" + "="*80)

AJUSTE Y TRANSFORMACIÓN

📊 Ajustando preprocessor con datos de entrenamiento...
   (IMPORTANTE: Solo usar train para evitar data leakage)
✓ Preprocessor ajustado exitosamente

🔄 Transformando datos de entrenamiento...
✓ X_train transformado: (1719, 38)

🔄 Transformando datos de evaluación...
✓ X_test transformado: (430, 38)

📈 Resumen de transformación:
   Features originales:    38
   Features transformados: 38
   Diferencia: +0

✓ Preprocessor ajustado exitosamente

🔄 Transformando datos de entrenamiento...
✓ X_train transformado: (1719, 38)

🔄 Transformando datos de evaluación...
✓ X_test transformado: (430, 38)

📈 Resumen de transformación:
   Features originales:    38
   Features transformados: 38
   Diferencia: +0



## 7. Guardado de Artefactos

Guardamos el preprocessor ajustado y los datasets transformados para uso en el siguiente paso del pipeline.

In [11]:
print("="*80)
print("GUARDANDO ARTEFACTOS")
print("="*80 + "\n")

# Crear directorios si no existen
artifacts_dir = Path("../../artifacts")
artifacts_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path("../../data/processed")
data_dir.mkdir(parents=True, exist_ok=True)

# 1. Guardar preprocessor ajustado
preprocessor_path = artifacts_dir / "preprocessor.joblib"
joblib.dump(preprocessor, preprocessor_path)
print(f"💾 Preprocessor guardado en: {preprocessor_path}")
print(f"   Tamaño: {preprocessor_path.stat().st_size / 1024:.2f} KB")

# 2. Guardar datasets transformados como CSV
X_train_df = pd.DataFrame(X_train_transformed)
X_test_df = pd.DataFrame(X_test_transformed)
y_train_df = pd.DataFrame(y_train).reset_index(drop=True)
y_test_df = pd.DataFrame(y_test).reset_index(drop=True)

X_train_path = data_dir / "X_train.csv"
X_test_path = data_dir / "X_test.csv"
y_train_path = data_dir / "y_train.csv"
y_test_path = data_dir / "y_test.csv"

X_train_df.to_csv(X_train_path, index=False)
X_test_df.to_csv(X_test_path, index=False)
y_train_df.to_csv(y_train_path, index=False)
y_test_df.to_csv(y_test_path, index=False)

print(f"\n💾 Datasets guardados en: {data_dir}")
print(f"   • X_train.csv: {X_train_path.stat().st_size / 1024:.2f} KB")
print(f"   • X_test.csv:  {X_test_path.stat().st_size / 1024:.2f} KB")
print(f"   • y_train.csv: {y_train_path.stat().st_size / 1024:.2f} KB")
print(f"   • y_test.csv:  {y_test_path.stat().st_size / 1024:.2f} KB")

# 3. Guardar metadatos
metadata = {
    "n_features_original": X_train.shape[1],
    "n_features_transformed": X_train_transformed.shape[1],
    "n_numeric_features": len(numeric_features),
    "n_categorical_features": len(nominal_features),
    "n_samples_train": X_train.shape[0],
    "n_samples_test": X_test.shape[0],
    "test_size": test_size,
    "random_state": random_state,
    "target_column": target_col,
    "features_created": [
        'Cholesterol_Ratio_LDL_HDL',
        'Cholesterol_Total_HDL_Ratio',
        'Mean_Arterial_Pressure',
        'Age_Squared',
        'Age_FH_Interaction',
        'CV_Risk_Score'
    ]
}

metadata_path = artifacts_dir / "feature_engineering_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n💾 Metadata guardado en: {metadata_path}")

print("\n" + "="*80)
print("✅ ARTEFACTOS GUARDADOS EXITOSAMENTE")
print("="*80)

GUARDANDO ARTEFACTOS

💾 Preprocessor guardado en: ..\..\artifacts\preprocessor.joblib
   Tamaño: 5.47 KB

💾 Datasets guardados en: ..\..\data\processed
   • X_train.csv: 1257.41 KB
   • X_test.csv:  314.63 KB
   • y_train.csv: 5.05 KB
   • y_test.csv:  1.27 KB

💾 Metadata guardado en: ..\..\artifacts\feature_engineering_metadata.json

✅ ARTEFACTOS GUARDADOS EXITOSAMENTE


## 8 Documentación de Decisiones de Preprocesamiento

### Decisiones Técnicas Justificadas

#### **Imputación de Valores Faltantes**

| Variable | Estrategia | Justificación |
|----------|-----------|--------------|
| **Numéricas** | Mediana | Robusta ante outliers, preserva distribución; preferible a media para features con sesgos |
| **Categóricas** | Valor más frecuente | Preserva modo, mantiene distribuciones de probabilidad |

**Alternativas consideradas y descartadas**:
- ❌ Eliminación listwise: Perdería muchas muestras
- ❌ Media para numéricas: Sensible a outliers en variables biomédicas
- ❌ Forward-fill: No aplicable (sin serie temporal)

#### **Escalado de Variables Numéricas (StandardScaler)**

```
x_scaled = (x - mean) / std_dev
```

**Justificación**:
- ✅ Algoritmos (regresión logística, SVM) sensibles a escala
- ✅ Facilita convergencia en gradient descent
- ✅ Features en escala comparable
- ✅ Mejor interpretabilidad de coeficientes

**Por qué StandardScaler y no MinMaxScaler**:
- StandardScaler es robusto ante outliers extremos en datos médicos
- No asume rango fijo (mejor para distribuciones no acotadas)
- Produce distribuciones aproximadamente normales

#### **Codificación Categórica (OneHotEncoder)**

**Justificación**:
- ✅ Variables nominales sin orden (género, tipo de síntoma)
- ✅ Evita asumir orden artificial
- ✅ Compatible con mayoría de ML algorithms

**Parámetro `handle_unknown='ignore'`**:
- Categorías nuevas en test → fila de ceros
- Alternativa `error` sería frágil en producción

#### **Separación Train-Test con Estratificación**

**Justificación**:
- ✅ Evita bias en distribución de clases
- ✅ Especialmente importante con clases desbalanceadas
- ✅ Garantiza train y test representen población similar

**Test Size = 20%**:
- ✅ Estándar en ML con datasets medianos (n > 1,000)
- ✅ Balance entre datos de entrenamiento y evaluación
- ✅ Configurable desde `config.json` para flexibilidad

#### **Sin Data Leakage**

**Implementación**:
```python
preprocessor.fit(X_train)      # ← Ajustar SOLO en train
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)  # ← Usar parámetros de train
```

**Crítico para evitar overfitting simulado**:
- ❌ Escalar con estadísticos de TODO el dataset → LEAKAGE
- ✅ Escalar con estadísticos solo de train → CORRECTO

### Parámetros del Pipeline

| Componente | Parámetro | Valor | Justificación |
|-----------|-----------|-------|---------------|
| Train-Test Split | `test_size` | 0.2 | 80-20 estándar |
| | `random_state` | 42 | Reproducibilidad |
| | `stratify` | sí | Mantener proporciones de clases |
| StandardScaler | `with_mean` | True | Centrar en 0 |
| | `with_std` | True | Escalar a varianza unitaria |
| OneHotEncoder | `sparse_output` | False | Matriz densa (compatible con más modelos) |
| | `handle_unknown` | 'ignore' | Robustez en producción |
| SimpleImputer | strategy (num) | 'median' | Robustez ante outliers |
| | strategy (cat) | 'most_frequent' | Mantiene modo |
